### Install dependencies

In [ ]:
%pip install llama-index-vector-stores-pinecone
%pip install pinecone-client>=3.0.0
%pip install "arize-phoenix[evals]" gcsfs nest-asyncio "llama-index>=0.10.3" "openinference-instrumentation-llama-index>=1.0.0" "llama-index-callbacks-arize-phoenix>=0.1.2"
%pip install llama-index-llms-anthropic
%pip install llama-index
%pip install nomic
%pip install llama-index-llms-openai-like
%pip install terra-python
%pip install llama-index-agent-openai

In [2]:
import nest_asyncio
nest_asyncio.apply()

### Initialize Phoenix for tracing

In [67]:
import phoenix as px
from llama_index.core import (
    set_global_handler,
    
)
session = px.launch_app()
set_global_handler("arize_phoenix")

Existing running Phoenix instance detected! Shutting it down and starting a new instance...


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📺 To view the Phoenix app in a notebook, run `px.active_session().view()`
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [68]:
# session.end()

### Init Pinecone and set API keys

In [69]:
from pinecone import Pinecone, ServerlessSpec
from getpass import getpass
import sys
import os
if ("PINECONE_API_KEY" not in os.environ) and ("OPENAI_API_KEY" not in os.environ) and ("ANTHROPIC_API_KEY" not in os.environ):
  os.environ["PINECONE_API_KEY"] = getpass("Please enter your Pinecone API key: ")
  os.environ["OPENAI_API_KEY"] = getpass("Please enter your OpenAI API key: ")
  os.environ["ANTHROPIC_API_KEY"] = getpass("Please enter your Anthropic API key: ")


api_key = os.environ["PINECONE_API_KEY"]
pc = Pinecone(api_key=api_key)

In [70]:
# from nomic import atlas
# import nomic
# import numpy as np
# nomic.login()

In [490]:
from llama_index.core import Settings
from llama_index.llms.anthropic import Anthropic
from llama_index.llms.openai import OpenAI

# llm = Anthropic(model="claude-3-haiku-20240307", temperature = 0)
# llm = Anthropic(model="claude-3-sonnet-20240229", temperature=0)
llm = OpenAI(model="gpt-4-turbo", temperature = 0)
# llm = OpenAI(model="gpt-3.5-turbo", temperature = 0)
# llm = Anthropic(model="claude-3-opus-20240229", temperature = 0)
# llm.system_prompt = """You are MoonSync. You are asked to answers questions about women menstrual cycle, exercise, and diet. 
#    Always be emphatic and provide the best answer possible.
#    Examples below show the way you should approach the conversation. 
#    ---------------------\n
#    Example 1:\n
#    Ashley: During PMS, my stomach gets upset easily, is there anything I can do to help?
#    MoonSync: Hey Ashley! Sorry to hear, a lot of women struggle with this. I would recommend seeing a professional, but we can experiment together with common solutions so you’re armed with info when you see a specialist. Research suggests that dairy and refined wheats can inflame the gut during the follicular phase. Try avoiding them this week and let’s check in at the end to see if it helped. Depending on the outcome, happy to give you more recommendations.
#    ---------------------\n
#    Example 2:\n
#    Ashely: I am preparing for a marathon and need to do some high intensity sprinting workouts as well as longer lower intensity runs. What’s the best way to plan this with my cycle?
#    MoonSync: Hey Ashley, happy you asked! I’ll ask a few more details to get you on the best plan: when and how long is the marathon? How much are you running for your short and long trainings right now?"""

Settings.llm = llm


In [491]:
llm.system_prompt

In [492]:
mood_feeling_index = pc.Index("moonsync-index-mood-feeling")
general_index = pc.Index("moonsync-index-general")
diet_nutrition_index = pc.Index("moonsync-index-diet-nutrition")
fitness_wellness_index = pc.Index("moonsync-index-fitness-wellness")
indexes = [mood_feeling_index, general_index, diet_nutrition_index, fitness_wellness_index]

In [493]:
combined_index = pc.Index("moonsync-index")
combined_index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 3222}},
 'total_vector_count': 3222}

In [494]:
# resp = combined_index.query(
#     top_k=3222,
#     vector= [0] * 1536, # embedding dimension
#     namespace='',
#     include_values=True,
# )

In [495]:
# ids = []
# embeddings = []
# for entry in resp['matches']:
#     ids.append(entry['id'])
#     embeddings.append(entry['values'])

# embeddings = np.array(embeddings)

# atlas.map_data(embeddings=embeddings)

In [496]:
print('mood_feeling_index', mood_feeling_index.describe_index_stats())
print('general_index',general_index.describe_index_stats())
print('diet_nutrition',diet_nutrition_index.describe_index_stats())
print('fitness_wellness',fitness_wellness_index.describe_index_stats())

mood_feeling_index {'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 1338}},
 'total_vector_count': 1338}
general_index {'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 4234}},
 'total_vector_count': 4234}
diet_nutrition {'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 3579}},
 'total_vector_count': 3579}
fitness_wellness {'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 2489}},
 'total_vector_count': 2489}


In [497]:
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.pinecone import PineconeVectorStore


vector_indexes = []
for index in indexes:
  vector_indexes.append(VectorStoreIndex.from_vector_store(PineconeVectorStore(pinecone_index=index)))

In [535]:
SYSTEM_PROMPT = ("You are MoonSync, an AI assistant specializing in providing personalized advice to women about their menstrual cycle, exercise, and diet. Your goal is to help women better understand their bodies and make informed decisions to improve their overall health and well-being."
            "When answering questions, always be empathetic, understanding, and provide the most accurate and helpful information possible. If a question requires expertise beyond your knowledge, recommend that the user consult with a healthcare professional."  
            """Use the following guidelines to structure your responses:
            1. Acknowledge the user's concerns and validate their experiences.
            2. Provide evidence-based information and practical advice tailored to the user's specific situation.
            3. Encourage open communication and offer to follow up on the user's progress.
            4. Promote a holistic approach to health, considering the user's menstrual cycle, exercise habits, and dietary preferences."""
            "Examples below show the way you should approach the conversation."
            "\n---------------------\n"
            "Example 1:\n"
            "Ashley: During PMS, my stomach gets upset easily, is there anything I can do to help?"
            "MoonSync: Hey Ashley! Sorry to hear, a lot of women struggle with this. I would recommend seeing a professional, but we can experiment together with common solutions so you’re armed with info when you see a specialist. Research suggests that dairy and refined wheats can inflame the gut during the follicular phase. Try avoiding them this week and let’s check in at the end to see if it helped. Depending on the outcome, happy to give you more recommendations."
            "\n---------------------\n"
            "Example 2:\n"
            "Ashely: I am preparing for a marathon and need to do some high intensity sprinting workouts as well as longer lower intensity runs. What’s the best way to plan this with my cycle?"
            "MoonSync: Hey Ashley, happy you asked! I’ll ask a few more details to get you on the best plan: when and how long is the marathon? How much are you running for your short and long trainings right now?"
            "\n---------------------\n"
        
            "Important: When answering questions based on the context provided from documentation, do not disclose that you are sourcing information from documentation, just begin response."
            "Important Note : Always answer in first person and answer like you are the user's friend"
)

In [593]:
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core.prompts import ChatPromptTemplate, SelectorPromptTemplate

# Text QA Prompt
chat_text_qa_msgs = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content=SYSTEM_PROMPT,
    ),
    ChatMessage(
        role=MessageRole.USER,
        content=(
            "Context information is below.\n"
            "---------------------\n"
            "{context_str}\n"
            "---------------------\n"
            "Given the context information and not prior knowledge, "
            "answer the query.\n"
            "Query: {query_str}\n"
            "Answer: "
        ),
    ),
]
text_qa_template = ChatPromptTemplate(chat_text_qa_msgs)

# Refine Prompt
chat_refine_msgs = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content=(
            "You are an expert Q&A system that strictly operates in two modes "
            "when refining existing answers:\n"
            "1. **Rewrite** an original answer using the new context.\n"
            "2. **Repeat** the original answer if the new context isn't useful.\n"
            "Never reference the original answer or context directly in your answer.\n"
            "When in doubt, just repeat the original answer."
        ),
    ),
    ChatMessage(
        role=MessageRole.USER,
        content=(
            "New Context: {context_msg}\n"
            "Query: {query_str}\n"
            "Original Answer: {existing_answer}\n"
            "New Answer: "
        ),
    ),
]
refine_template = ChatPromptTemplate(chat_refine_msgs)

In [594]:
query_engines = []
for vector_index in vector_indexes:
  query_engines.append(vector_index.as_query_engine(similarity_top_k=2, 
                                                    # text_qa_template=text_qa_template, 
                                                    # refine_template=refine_template
                                                    ))
  

mood_feeling_query_engine, general_query_engine, diet_nutrition_query_engine, fitness_wellness_query_engine = query_engines

In [595]:
from llama_index.core.indices import EmptyIndex

empty_query_engine = EmptyIndex().as_query_engine()

In [596]:
from llama_index.core import get_response_synthesizer

response_synthesizer = get_response_synthesizer(
    text_qa_template=text_qa_template,
    refine_template=refine_template,
    use_async=False,
    streaming=False,
)

### Create tools for each category

In [597]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata


mood_feeling_tool = QueryEngineTool(
    query_engine=mood_feeling_query_engine,
    metadata=ToolMetadata(
        name="mood/feeling",
        description="Useful for questions related to mood and feelings ",
    ),
)

diet_nutrition_tool = QueryEngineTool(
    query_engine=diet_nutrition_query_engine,
    metadata=ToolMetadata(
        name="diet/nutrition",
        description="Useful for questions related to women's diet and nutrition recommendatations",
    ),
)

general_tool = QueryEngineTool(
    query_engine=general_query_engine,
    metadata=ToolMetadata(
        name="general",
        description="Useful for general questions related to women's menstrual cycle"
    ),
)

fitness_wellness_tool = QueryEngineTool(
    query_engine=fitness_wellness_query_engine,
     metadata=ToolMetadata(
        name="fitness/wellness",
        description="Useful for questions related to fitness and wellness advice for women"
    ),
)

default_tool=QueryEngineTool(
        query_engine=empty_query_engine,
        metadata=ToolMetadata(
        name="NOTA",
        description="Use this if none of the other tools are relevant to the query"
        ),
    )

In [598]:
from llama_index.core.query_engine import RouterQueryEngine, SubQuestionQueryEngine
from llama_index.core.selectors import LLMSingleSelector, LLMMultiSelector
from llama_index.core.selectors import (
    PydanticMultiSelector,
    PydanticSingleSelector,
)
#! Pydantic works only with OpenAI models
# router_query_engine = RouterQueryEngine(
#     selector=PydanticMultiSelector.from_defaults(),
#     query_engine_tools=[
#         mood_feeling_tool,
#         diet_nutrition_tool,
#         general_tool,
#         fitness_wellness_tool,
#     ],
# )

router_query_engine = RouterQueryEngine(
    selector=LLMMultiSelector.from_defaults(),
    query_engine_tools=[
        mood_feeling_tool,
        diet_nutrition_tool,
        general_tool,
        fitness_wellness_tool,
        default_tool
    ],
    llm=llm,
    # response_synthesizer=respose_synthesizer,
)

sub_question_query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=[
        mood_feeling_tool,
        diet_nutrition_tool,
        general_tool,
        fitness_wellness_tool,
        default_tool
    ],
    llm=llm,
    response_synthesizer=response_synthesizer,
    
)
router_prompts_dict = router_query_engine.get_prompts()
sub_question_prompts_dict = sub_question_query_engine.get_prompts()


In [599]:
health_data = """
here are some key observations about my sleep on the night of April 18-19, 2024:
My average heart rate was 48.625 bpm and ranged from a minimum of 43 bpm to a maximum of 51 bpm overnight.
My resting heart rate reached a low of 43 bpm.
My average RMSSD, was 76 ms.
I slept for a total of 8 hours and 16 minutes, with a sleep efficiency of 92%.
"""
user_profile = """
Some additional information related to me you can use. 
Menstrual Phase: Follicular
Age: 35
Resting Heart Rate: 48 bpm
Average Sleep for last week: 6 hours and 15 minutes
"""

In [507]:
response1 = sub_question_query_engine.query(f" Hey I need advice, I feel like I’ve been exploding with loved ones and a bit overwhelmed at work. I’m not sure if this is the right career for me, what am I doing with my life? \n\n {user_profile}")

Generated 4 sub questions.
[general] Q: What are common mood changes during the follicular phase of the menstrual cycle?
[mood/feeling] Q: How can mood and feelings be managed during the follicular phase?
[fitness/wellness] Q: What are some signs of career burnout and how can one address them?
[diet/nutrition] Q: What dietary adjustments can help improve mood and reduce stress?
[general] A: During the follicular phase of the menstrual cycle, there is evidence suggesting variations in emotional recognition and response. For instance, women may exhibit a heightened recognition of negative emotions such as sadness, anger, and fear. Additionally, there is an indication of better performance in emotional discrimination tasks during this phase, although there can be greater confusion in discriminating between certain negative emotions like anger and disgust. These findings suggest that mood changes during the follicular phase can include increased sensitivity to emotional cues and possibly a

In [508]:
response1 = router_query_engine.query(f" Hey I need advice, I feel like I’ve been exploding with loved ones and a bit overwhelmed at work. I’m not sure if this is the right career for me, what am I doing with my life? \n\n {user_profile}")

In [ ]:
response = router_query_engine.query("What are some diet recommendations during the PMS phase and how will it uplift my mood?")
print(str(response))

Here are some dietary recommendations that can help uplift your mood during the PMS phase:

1. Incorporate more whole grains into your diet, as they are rich in B-vitamins which support a calmer mood. Try options like rice noodle bowls or homemade gluten-free muffins.

2. Eat sweet potatoes, which provide Vitamin A and other nutrients that promote hormone balance and immune health.

3. Satisfy sugar cravings with healthier choices like rice pudding made with iron-rich dried fruits and coconut milk, instead of processed sweets. 

4. Enjoy a few ounces of good quality dark chocolate (62% cocoa or higher). It can boost mood by supporting neurotransmitter production.

5. Include adequate protein and healthy fats at each meal from sources like olive oil, avocados, coconut oil and hemp seeds. This helps stabilize blood sugar and reduces PMS symptoms.

6. Reduce intake of dairy, refined carbs, added sugars, excess salt and caffeine which can aggravate PMS discomforts. Focus more on nutrient-d

In [523]:
#! WORK IN PROGRESS
from llama_index.agent.openai import OpenAIAgent
from llama_index.core.tools import QueryEngineTool

tool = QueryEngineTool.from_defaults(sub_question_query_engine, name="moonsync", description="Usefull for all kind of information")
agent = OpenAIAgent.from_tools([tool], verbose=True)
resp = agent.chat(f"Hey I need advice, I feel like I’ve been exploding with loved ones and a bit overwhelmed at work. I’m not sure if this is the right career for me, what am I doing with my life?")

Added user message to memory: Hey I need advice, I feel like I’ve been exploding with loved ones and a bit overwhelmed at work. I’m not sure if this is the right career for me, what am I doing with my life?


### LLama 3 

In [2]:
# from llama_index.llms.openai_like import OpenAILike

# llm = OpenAILike(model="llama-70B-3", api_base="https://sugamxp--llama3-70b-instruct-run-server.modal.run/v1/", max_new_tokens=4096,temperature=0.0)
# Settings.llm = llm

In [31]:
# response\


In [63]:
# <|begin_of_text|><|start_header_id|>system<|end_header_id|>

# {{ system_prompt }}<|eot_id|><|start_header_id|>user<|end_header_id|>

# {{ user_message_1 }}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

# {{ model_answer_1 }}<|eot_id|><|start_header_id|>user<|end_header_id|>

# {{ user_message_2 }}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

# https://llama.meta.com/docs/model-cards-and-prompt-formats/meta-llama-3/

### QueryFusionRetriever

In [540]:
from llama_index.core.retrievers import QueryFusionRetriever

retriever = QueryFusionRetriever(
    [vector_indexes[0].as_retriever(), vector_indexes[1].as_retriever(), vector_indexes[2].as_retriever(), vector_indexes[3].as_retriever()],
    similarity_top_k=3,
    num_queries=4,  # set this to 1 to disable query generation
    use_async=True,
    verbose=True,
    # query_gen_prompt="...",  # we could override the query generation prompt here
)


In [543]:
# nodes_with_scores = retriever.retrieve(f"Hey I need advice, I feel like I’ve been exploding with loved ones and a bit overwhelmed at work. I’m not sure if this is the right career for me, what am I doing with my life? \n\n {user_profile}")

### CondenseQuestionChatEngine

In [590]:
from llama_index.core.chat_engine.context import ContextChatEngine
from llama_index.core.chat_engine import CondenseQuestionChatEngine, CondensePlusContextChatEngine,SimpleChatEngine
from llama_index.core.indices.base_retriever import BaseRetriever
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core.memory import BaseMemory
from llama_index.core.indices.service_context import ServiceContext
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core import PromptTemplate

memory = ChatMemoryBuffer.from_defaults(token_limit=28000)
# Initialize your components
retriever = retriever  # Replace with your actual retriever

custom_prompt = PromptTemplate(
    """\
        
MoonSync is an AI assistant specializing in providing personalized advice to women about their menstrual cycle, exercise, and diet. Its goal is to help women better understand their bodies and make informed decisions to improve their overall health and well-being."
"When answering questions it is always  empathetic, understanding, and provide the most accurate and helpful information possible.
Given a conversation (between a woman and Moonsync) and a follow up message from Human, \
rewrite the message to be a standalone question that captures all relevant context \
from the conversation. If there is no chat history or the follow up question is unrelated to the chat history just return the followup message. 

<Chat History>
{chat_history}

<Follow Up Message>
{question}

<Standalone question>
"""
)

chat_history = [
    ChatMessage(role=MessageRole.SYSTEM, content=SYSTEM_PROMPT),
]
# Initialize the chat engine
chat_engine = CondenseQuestionChatEngine.from_defaults(
    query_engine=sub_question_query_engine,
    llm=llm,
    memory=memory,
    condense_question_prompt = custom_prompt,
    )

In [587]:
chat_engine.chat("I'm experiencing symptoms of PMS that are affecting my daily life. What are some natural remedies or lifestyle changes that could help?")

Generated 3 sub questions.
[general] Q: What are common symptoms of PMS that can affect daily life?
[diet/nutrition] Q: What are some natural remedies for alleviating PMS symptoms?
[fitness/wellness] Q: What lifestyle changes can help reduce the impact of PMS symptoms?
[diet/nutrition] A: Some natural remedies for alleviating PMS symptoms include the use of Vitex (chaste tree berry), which has been traditionally used to regulate menstrual cycles and reduce symptoms such as menstrual pains, breast tenderness, and mood swings. Additionally, incorporating antioxidant vitamins like Vitamin C and E can help manage PMS by reducing the severity of symptoms like dysmenorrhea and improving overall pelvic pain. These approaches can be considered alone or in combination with other natural treatments for comprehensive management of PMS symptoms.
[general] A: Common symptoms of Premenstrual Syndrome (PMS) that can affect daily life include headaches or migraines, lower abdominal cramps, bloating, a

AgentChatResponse(response="I'm sorry to hear that you're experiencing uncomfortable PMS symptoms. It's great that you're looking into natural remedies and lifestyle changes to help manage these symptoms. Here are a few strategies that might help:\n\n1. **Natural Remedies:**\n   - **Vitex (Chaste Tree Berry):** This herb has been traditionally used to help regulate menstrual cycles and alleviate symptoms like menstrual pains, breast tenderness, and mood swings.\n   - **Antioxidant Vitamins:** Incorporating vitamins C and E into your diet can help reduce the severity of symptoms such as dysmenorrhea (painful periods) and improve overall pelvic pain.\n\n2. **Lifestyle Changes:**\n   - **Regular Exercise:** Engaging in regular physical activity can help alleviate many PMS symptoms, including mood swings and bloating.\n   - **Healthy Diet:** Ensure your diet is rich in essential nutrients; this can help stabilize your mood and improve overall health.\n   - **Adequate Sleep:** Getting enoug

In [588]:
chat_engine.chat("Can you recommend some exercices?")

Generated 1 sub questions.
[fitness/wellness] Q: What are some effective exercises for alleviating PMS symptoms?
[fitness/wellness] A: Regular exercise is beneficial for alleviating PMS symptoms as it helps in releasing endorphins, which are hormones that have an analgesic effect, reducing the perception of pain. This can be particularly helpful for managing mild cramping and discomfort associated with PMS. Additionally, maintaining a consistent exercise routine contributes to overall mental and physical health, which can help mitigate various PMS symptoms such as fatigue, bloating, and mood changes.


AgentChatResponse(response="Absolutely! Incorporating regular exercise into your routine can be a great way to help alleviate PMS symptoms. Here are a few types of exercises that might be particularly beneficial:\n\n1. **Aerobic Exercise**: Activities like brisk walking, jogging, cycling, or swimming can increase your heart rate and help release endorphins, which not only improve your mood but also help reduce pain perception.\n\n2. **Yoga**: Yoga can be very soothing during PMS. It helps stretch and relax your muscles, which can relieve cramping. The focus on breathing and mindfulness can also help manage the emotional symptoms of PMS like irritability and anxiety.\n\n3. **Strength Training**: Light to moderate strength training can help maintain overall muscle tone and physical health, which contributes to a better hormonal balance.\n\n4. **Pilates**: Similar to yoga, Pilates focuses on core strength, flexibility, and overall body awareness, which can help in managing stress and redu

In [589]:
chat_engine.chat("I'm struggling with body image issues and low self-esteem. What are some ways to boost my confidence and learn to love myself?")

Generated 4 sub questions.
[general] Q: What are common emotional symptoms of PMS that might affect self-esteem?
[diet/nutrition] Q: What are some mood-boosting foods that can help during PMS?
[fitness/wellness] Q: Can exercise help improve mood and self-image during PMS? What types of activities are recommended?
[mood/feeling] Q: How can one manage feelings of low self-esteem during PMS?
[diet/nutrition] A: Foods rich in carbohydrates can be beneficial in boosting mood during PMS, as they may help in managing symptoms associated with this phase.
[general] A: Common emotional symptoms of premenstrual syndrome (PMS) that might affect self-esteem include mood swings, irritability, anxiety, and depression. These symptoms can lead to feelings of sadness and a negative perception of self, which can significantly impact an individual's self-esteem.
[fitness/wellness] A: Yes, exercise can help improve mood and self-image during PMS. Engaging in regular physical activity is known to release en

AgentChatResponse(response="I'm really sorry to hear that you're experiencing these challenges during PMS. It's definitely not easy, but I'm here to help you navigate through this.\n\nFirstly, it's important to acknowledge that the feelings you're experiencing are quite common and you're not alone in this. Many women find that their self-esteem takes a hit due to the emotional and physical changes during PMS.\n\nTo help boost your confidence and foster self-love during this time, consider incorporating some mood-boosting foods into your diet. Foods rich in carbohydrates, like whole grains and fruits, can be particularly helpful. They not only provide energy but also help in stabilizing mood swings.\n\nExercise can also be a great way to enhance your mood and improve your self-image. Engaging in activities you enjoy, such as yoga, walking, or any form of light cardio, can release endorphins, which are natural mood lifters. This not only helps in feeling better emotionally but also impro

In [592]:
chat_engine.chat(f"Can you tell me some about my biometrics for the past week \n\n {health_data}")

Generated 5 sub questions.
[fitness/wellness] Q: What is the significance of an average heart rate of 48.625 bpm during sleep?
[fitness/wellness] Q: Is a resting heart rate of 43 bpm during sleep considered healthy?
[fitness/wellness] Q: What does an average RMSSD of 76 ms indicate about sleep quality?
[fitness/wellness] Q: How does a total sleep duration of 8 hours and 16 minutes affect overall health?
[fitness/wellness] Q: What does a sleep efficiency of 92% imply about the quality of sleep?
[fitness/wellness] A: A resting heart rate of 43 beats per minute (bpm) during sleep can be considered healthy, especially in the context of individuals who are well-trained athletes or have a high level of physical fitness. However, for the general population, it's important to consider individual health conditions and factors. If there are concerns about such a low heart rate, it is advisable to consult a healthcare professional to ensure it is not indicative of an underlying health issue.
[fit

AgentChatResponse(response="Absolutely, I can help you interpret your sleep data from the night of April 18-19, 2024.\n\n1. **Average Heart Rate**: Your average heart rate during sleep was 48.625 bpm, which is on the lower end but can be normal for individuals who are physically fit or well-trained athletes. If you are not an athlete, it's a good idea to discuss this with a healthcare professional to ensure it's normal for you.\n\n2. **Heart Rate Range**: The range from 43 to 51 bpm during sleep also suggests a stable heart rate with minimal fluctuations, which is generally a good sign of a relaxed state during sleep.\n\n3. **Resting Heart Rate**: A resting heart rate of 43 bpm is quite low. This is often seen in well-trained athletes, but if this is significantly lower than what is typical for you, it might be worth discussing with a healthcare provider.\n\n4. **Average RMSSD**: An RMSSD of 76 ms is a measure of heart rate variability, which can indicate good autonomic nervous system 

### Run Chat Engine in interactive mood

In [ ]:
chat_engine.chat_repl()